# 📚 RAG 시스템 완전 정복: 을GPT 0주차 

을GPT ai및rag팀을 위한 노트북입니다.
공유를 금지합니다. 

## 🎯 학습 목표

이 노트북을 통해 다음을 배울 수 있습니다:

### 📖 이론적 이해
- ✅ RAG(Retrieval-Augmented Generation)의 핵심 개념
- ✅ 임베딩과 벡터 검색의 원리
- ✅ 프롬프트 엔지니어링 기법
- ✅ AI 시스템 평가 및 개선 방법

### 🛠️ 실무적 기술
- ✅ Google Gemini API 활용법
- ✅ LangChain 프레임워크 사용법
- ✅ 벡터 데이터베이스(Chroma) 구축
- ✅ 실제 문서를 활용한 QA 시스템 개발

## 📋 노트북 구성

| 단계 | 제목 | 주요 내용 | 소요 시간 |
|------|------|-----------|-----------|
| **1️⃣** | 환경 설정 | 라이브러리 import, API 설정 | 5분 |
| **2️⃣** | 문서 로딩 | 텍스트 파일 불러오기 | 5분 |
| **3️⃣** | 텍스트 분할 | 문서를 적절한 크기로 나누기 | 10분 |
| **4️⃣** | 임베딩 생성 | 텍스트를 벡터로 변환 | 10분 |
| **5️⃣** | 벡터DB 구축 | 검색 가능한 데이터베이스 생성 | 10분 |
| **6️⃣** | RAG 체인 구성 | 검색+생성 파이프라인 구축 | 15분 |
| **7️⃣** | 프롬프트 최적화 | 답변 품질 향상 | 15분 |
| **8️⃣** | 성능 평가 | 시스템 테스트 및 개선 | 10분 |

**총 소요 시간: 약 80분** ⏰

## 🚀 시작하기 전에

### 📋 사전 요구사항

**필수 지식**:
- 기본적인 Python 문법
- Jupyter Notebook 사용법
- API 개념에 대한 이해

**준비사항**:
- Google AI Studio 계정 및 API 키
- Python 환경 (Python 3.8+)
- 필요한 패키지 설치

### 💡 학습 방법

1. **순서대로 실행**: 셀을 위에서부터 차례대로 실행하세요
2. **마크다운 정독**: 각 코드 전에 있는 설명을 꼼꼼히 읽어보세요
3. **실험해보기**: 매개변수를 바꿔가며 결과 변화를 관찰하세요
4. **질문하기**: 이해가 안 되는 부분은 주저하지 말고 질문하세요

**이제 AI의 미래 기술인 RAG를 함께 배워봅시다!** 🚀

# 🤖 Google Gemini API를 활용한 RAG(Retrieval-Augmented Generation) 시스템 구축

## 📚 RAG란 무엇인가요?

**RAG(Retrieval-Augmented Generation)**는 대화형 AI의 새로운 패러다임입니다.

### 🔍 기존 AI와 RAG의 차이점

| 구분 | 기존 AI 모델 | RAG 시스템 |
|------|-------------|------------|
| **정보 소스** | 훈련 데이터에만 의존 | 외부 문서 + 훈련 데이터 |
| **최신성** | 훈련 시점까지의 정보 | 실시간 문서 업데이트 가능 |
| **정확성** | 할루시네이션 발생 가능 | 문서 기반의 정확한 답변 |
| **투명성** | 답변 근거 불명확 | 참고 문서 출처 제공 |
| **전문성** | 일반적인 지식 | 특정 도메인 전문 지식 |

### 🎯 RAG의 동작 원리

1. **📄 문서 준비**: 전문 문서들을 시스템에 입력
2. **✂️ 문서 분할**: 큰 문서를 작은 청크(조각)로 나눔
3. **🧮 임베딩 변환**: 텍스트를 벡터(숫자 배열)로 변환
4. **💾 벡터 저장**: 벡터들을 데이터베이스에 저장
5. **🔍 유사도 검색**: 질문과 관련된 문서 조각 찾기
6. **🤖 답변 생성**: 찾은 문서와 질문을 AI에게 제공하여 답변 생성

### 💡 이 노트북에서 학습할 내용

- Google Gemini API 설정 방법
- 문서 로딩 및 전처리 기술
- 임베딩과 벡터 데이터베이스 이해
- RAG 체인 구성 및 최적화
- 실제 질의응답 시스템 구현

---

## 🛠️ 사전 준비사항

### 1. API 키 설정
```bash
# .env 파일에 다음 내용 추가
GOOGLE_API_KEY=your_google_api_key_here
```

### 2. 필요한 라이브러리
- `langchain`: RAG 시스템 구축 프레임워크
- `langchain-google-genai`: Google Gemini API 연동
- `chromadb`: 벡터 데이터베이스
- `python-dotenv`: 환경변수 관리

### 3. 학습 데이터
- `data/을지대_학업성적처리규정.txt`: 예시 문서

---

**🚀 이제 실제 RAG 시스템을 단계별로 구축해보겠습니다!**

## 🔍 RAG란 무엇인가요?

**RAG (Retrieval-Augmented Generation)**는 대화형 AI의 한계를 극복하기 위한 혁신적인 기술입니다.

### 📊 기존 LLM vs RAG 비교

| 특징 | 기존 LLM | RAG 시스템 |
|------|----------|------------|
| **정보 소스** | 훈련 데이터만 사용 | 외부 문서 + 훈련 데이터 |
| **최신 정보** | 훈련 시점까지만 | 실시간 문서 검색 가능 |
| **정확성** | 할루시네이션 발생 가능 | 근거 기반 답변 |
| **맞춤화** | 어려움 | 도메인 특화 가능 |

### 🏗️ RAG 시스템 구조

```
1. 📄 문서 수집 → 2. ✂️ 텍스트 분할 → 3. 🧮 임베딩 생성 → 4. 💾 벡터DB 저장
                                                    ↓
6. 🤖 AI 답변 생성 ← 5. 🔍 유사 문서 검색 ← 사용자 질문
```

### 💡 RAG가 중요한 이유

1. **정확성 향상**: 실제 문서를 기반으로 답변 생성
2. **최신 정보**: 새로운 문서 추가만으로 지식 업데이트
3. **투명성**: 답변의 근거가 되는 문서 확인 가능
4. **비용 효율성**: 전체 모델 재훈련 없이 지식 확장

이제 실제 코드로 RAG 시스템을 구축해보겠습니다! 🚀

# 1️⃣ 라이브러리 Import 및 초기 설정

## 📦 필요한 라이브러리들

RAG 시스템을 구축하기 위해 다음 라이브러리들을 사용합니다:

### 🔧 기본 유틸리티
- **`os`**: 운영체제 관련 기능 (파일 경로, 환경변수 등)
- **`dotenv`**: `.env` 파일에서 환경변수를 안전하게 로드

### 📄 문서 처리
- **`TextLoader`**: 텍스트 파일을 불러오는 도구
- **`RecursiveCharacterTextSplitter`**: 문서를 작은 조각으로 나누는 도구

### 🗄️ 벡터 데이터베이스
- **`Chroma`**: 임베딩 벡터를 저장하고 검색하는 데이터베이스

### 🤖 Google Gemini API
- **`ChatGoogleGenerativeAI`**: Google Gemini 언어모델
- **`GoogleGenerativeAIEmbeddings`**: Google 임베딩 모델
- **`RetrievalQA`**: 검색 기반 질의응답 체인

## 🔐 환경변수 설정의 중요성

```python
load_dotenv()  # .env 파일에서 GOOGLE_API_KEY 로드
```

이 코드는 프로젝트 루트의 `.env` 파일에서 API 키를 자동으로 읽어옵니다. 
API 키를 코드에 직접 작성하지 않아 보안을 유지할 수 있습니다.

## 🌟 Google Gemini API 소개

### 왜 Google Gemini API를 사용하나요?

**Google Gemini**는 Google의 최신 멀티모달 AI 모델로, RAG 시스템 구축에 매우 적합합니다.

### 🎯 Gemini API의 주요 장점

| 특징 | 설명 | RAG에서의 활용 |
|------|------|---------------|
| **고품질 임베딩** | `text-embedding-004` 모델 | 문서를 벡터로 변환 |
| **강력한 언어 모델** | `gemini-1.5-flash` | 검색된 문서 기반 답변 생성 |
| **한국어 지원** | 뛰어난 한국어 이해 능력 | 한국어 문서 처리에 최적 |
| **비용 효율성** | 합리적인 API 가격 | 실험 및 학습에 적합 |

### 🔧 필요한 구성 요소

1. **임베딩 모델**: 텍스트를 숫자 벡터로 변환
   - 문서와 질문을 같은 벡터 공간에 매핑
   - 유사도 계산으로 관련 문서 찾기

2. **언어 모델**: 검색된 정보를 바탕으로 답변 생성
   - 자연스러운 한국어 답변 생성
   - 문맥을 이해하고 적절한 응답 제공

### 📋 사전 준비사항

- slack으로 API키를 제공해드렸습니다. 
- 환경변수 설정 또는 직접 입력
- 필요한 Python 패키지 설치

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA

# .env 파일에서 환경 변수 로드
load_dotenv()

print("✅ Google Gemini API 전용 RAG 시스템")
print("   - Google Gemini LLM (gemini-1.5-flash)")
print("   - Google 임베딩 (text-embedding-004)")
print("   - 환경 변수 로드 완료")

# 문서 불러오기
loader = TextLoader("data/을지대_학업성적처리규정.txt", encoding="utf-8")
documents = loader.load()
print(f"\n📄 문서 로딩 완료: {len(documents)}개 문서")

c:\Users\geon1\anaconda3\envs\langgraph_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\geon1\anaconda3\envs\langgraph_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Google Gemini API 전용 RAG 시스템
   - Google Gemini LLM (gemini-1.5-flash)
   - Google 임베딩 (text-embedding-004)
   - 환경 변수 로드 완료

📄 문서 로딩 완료: 1개 문서


In [2]:
# 2. 문서 쪼개기
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

# 2️⃣ 문서 로딩 및 전처리

## 📄 문서 로딩이란?

RAG 시스템의 첫 번째 단계는 **지식 베이스가 될 문서들을 시스템에 로드**하는 것입니다.

### 🎯 문서 로딩의 목적

- **외부 지식 활용**: AI가 알지 못하는 최신 정보나 전문 지식 제공
- **도메인 특화**: 특정 분야(예: 대학 규정)에 특화된 정보 활용
- **정확성 향상**: 실제 문서를 기반으로 한 신뢰할 수 있는 답변

### 📂 지원하는 파일 형식

| 형식 | Loader | 특징 |
|------|--------|------|
| **TXT** | `TextLoader` | 일반 텍스트 파일 |
| **PDF** | `PyPDFLoader` | PDF 문서 |
| **Word** | `Docx2txtLoader` | Word 문서 |
| **웹페이지** | `WebBaseLoader` | HTML 페이지 |
| **CSV** | `CSVLoader` | 표 형태 데이터 |

### 📖 이번 예제에서 사용할 문서

- **을지대 학칙**: 대학의 기본 규정
- **학업성적처리규정**: 성적 관련 세부 규칙

이 문서들을 통해 대학 규정에 대한 질문답변 시스템을 만들어보겠습니다!

# 2️⃣ 문서 분할 (Text Splitting)

## 🔪 문서를 왜 쪼개야 할까요?

RAG 시스템에서 문서 분할은 매우 중요한 단계입니다:

### ❌ 문서를 통째로 사용할 때의 문제점
1. **메모리 한계**: 대용량 문서는 AI 모델의 입력 길이 제한 초과
2. **검색 정확도 저하**: 관련 없는 정보가 섞여 정확한 검색 어려움
3. **처리 속도 느림**: 큰 문서 처리에 많은 시간 소요

### ✅ 문서 분할의 장점
1. **정확한 검색**: 질문과 가장 관련된 부분만 찾아서 사용
2. **효율적 처리**: 작은 조각들로 빠른 검색 가능
3. **메모리 효율성**: AI 모델의 제한된 컨텍스트 윈도우 내에서 동작

## ⚙️ 분할 매개변수 이해하기

```python
RecursiveCharacterTextSplitter(
    chunk_size=500,      # 각 조각의 최대 글자 수
    chunk_overlap=50     # 인접한 조각 간 겹치는 글자 수
)
```

- **`chunk_size`**: 한 조각의 크기 (너무 작으면 문맥 손실, 너무 크면 검색 부정확)
- **`chunk_overlap`**: 조각 간 겹침 (문맥 연속성 유지, 정보 손실 방지)

In [3]:
# 개선된 텍스트 분할 (한국어 문서에 최적화)
from langchain.text_splitter import CharacterTextSplitter

def smart_text_splitter(documents):
    """의미 단위로 문서를 분할하는 개선된 함수"""
    korean_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,  # 더 작은 청크 크기
        chunk_overlap=100,  # 더 큰 오버랩
        separators=[
            "\n\n",  # 단락 구분
            "\n",    # 줄바꿈
            "。",     # 마침표
            ".",
            " ",     # 공백
            ""
        ],
        keep_separator=True
    )
    return korean_splitter.split_documents(documents)

# 개선된 분할 적용
improved_docs = smart_text_splitter(documents)
print(f"기존 문서 수: {len(docs)}")
print(f"개선된 문서 수: {len(improved_docs)}")
print(f"첫 번째 청크 예시:")
print(improved_docs[0].page_content[:200] + "...")

기존 문서 수: 10
개선된 문서 수: 20
첫 번째 청크 예시:
학업성적처리규정
제정 2007. 3. 1.
개정 2015. 3. 1.
개정 2017. 3. 1.
개정 2017. 9. 1.
개정 2018. 9. 1.
개정 2020. 3. 1.
개정 2022. 2. 1.
개정 2023. 3. 1.
개정 2024. 3. 1.
개정 2024. 4. 1.
개정 2025. 1.15.
제1조(목적) 이 규정은 을지대학교(이하 “본교...


### 📊 텍스트 분할 결과 분석

위 코드에서 **기존 문서 수**와 **개선된 문서 수**를 비교해보세요:

#### 🔍 분할 품질 체크리스트

✅ **좋은 분할의 특징**:
- 각 청크가 완전한 의미 단위를 포함
- 중요한 정보가 여러 청크에 나뉘지 않음
- 청크 간 적절한 중복으로 문맥 연결

❌ **피해야 할 분할**:
- 문장 중간에서 끊어짐
- 너무 작아서 의미가 불분명
- 너무 커서 관련 없는 내용 포함

#### 💡 분할 최적화 팁

```python
# 한국어 문서에 최적화된 구분자 순서
separators=[
    "\n\n",     # 단락 구분 (최우선)
    "\n",       # 줄바꿈
    "。",        # 마침표 (한국어)
    ".",        # 마침표 (영어)
    "?", "!",   # 물음표, 감탄표
    ";", ",",   # 세미콜론, 쉼표
    " ",        # 공백
    ""          # 최후의 수단
]
```

**다른 언어별 최적화**:
- **영어**: `["\n\n", "\n", ".", "?", "!", ";", ",", " ", ""]`
- **중국어**: `["\n\n", "\n", "。", "？", "！", "；", "，", " ", ""]`
- **일본어**: `["\n\n", "\n", "。", "？", "！", "；", "、", " ", ""]`

# 3️⃣ 한국어 최적화 문서 분할

## 🇰🇷 한국어 문서 처리의 특별함

영어와 달리 한국어는 고유한 특성이 있어 특별한 처리가 필요합니다:

### 📝 한국어 특성
- **교착어**: 조사와 어미가 붙어 의미 변화
- **띄어쓰기**: 영어와 다른 띄어쓰기 규칙
- **문장 구조**: 주어-목적어-동사(SOV) 구조
- **문서 형식**: 조항, 항, 호 등의 특별한 구조

## ⚡ 개선된 분할 전략

### 🔧 매개변수 최적화
```python
chunk_size=300     # 기존 500 → 300 (한국어는 더 작은 단위로)
chunk_overlap=100  # 기존 50 → 100 (문맥 보존 강화)
```

### 📐 한국어 구분자 우선순위
```python
separators=[
    "\n\n",    # 단락 구분 (가장 우선)
    "\n",      # 줄바꿈
    "。",       # 한국어 마침표
    ".",       # 영어 마침표
    " ",       # 공백
    ""         # 마지막 수단 (글자 단위 분할)
]
```

이 순서대로 문서를 나누어 의미 단위를 최대한 보존합니다.

# 3️⃣ 텍스트 분할 (Text Splitting)

## ✂️ 왜 텍스트를 분할해야 할까요?

긴 문서를 그대로 사용하면 여러 문제가 발생합니다:

### 🚫 문제점들

| 문제 | 설명 | 해결책 |
|------|------|--------|
| **토큰 제한** | AI 모델의 입력 길이 제한 | 적절한 크기로 분할 |
| **검색 정확도** | 관련 없는 내용이 섞임 | 의미 단위로 분할 |
| **처리 속도** | 큰 텍스트는 처리가 느림 | 작은 청크로 나누기 |

### 🛠️ 분할 전략

**RecursiveCharacterTextSplitter**의 동작 원리:
1. **문단 단위** 분할 시도 (`\n\n`)
2. **문장 단위** 분할 시도 (`\n`)
3. **단어 단위** 분할 시도 (` `)
4. **글자 단위** 최종 분할

### ⚙️ 주요 매개변수

- **chunk_size**: 각 청크의 최대 크기 (예: 1000자)
- **chunk_overlap**: 청크 간 중복 영역 (예: 200자)
  - 중복을 두는 이유: 문맥 연결성 유지

### 💡 청크 크기 선택 가이드

| 청크 크기 | 장점 | 단점 | 적합한 용도 |
|-----------|------|------|-------------|
| **작음 (500자)** | 정확한 검색 | 문맥 부족 | FAQ, 간단한 정보 |
| **중간 (1000자)** | 균형 잡힌 성능 | - | 일반적인 문서 |
| **큼 (2000자)** | 풍부한 문맥 | 노이즈 증가 | 복잡한 설명서 |

## 📊 분할 결과 비교
실행 후 다음을 확인할 수 있습니다:
- 기존 방식 vs 개선된 방식의 조각 개수 차이
- 더 세밀하고 의미 있는 단위로 분할된 결과

# Google Gemini 임베딩 모델 설정

Google Gemini API를 사용한 고품질 임베딩 시스템입니다.

## 📚 Google Embedding Models:

1. **text-embedding-004** (최신, 사용 중)
   - 768 차원
   - 다국어 지원 (한국어 우수)
   - 최고 성능
   - 의미 검색에 최적화

2. **text-embedding-gecko@003**
   - 768 차원  
   - 안정적인 성능

3. **textembedding-gecko@001**
   - 768 차원
   - 이전 버전

# 4️⃣ 임베딩(Embedding) 이해하기

## 🧮 임베딩이란?

**임베딩**은 텍스트를 컴퓨터가 이해할 수 있는 숫자 벡터로 변환하는 기술입니다.

### 🔄 변환 과정
```
"재수강 규정" → [0.123, -0.456, 0.789, ...] (768개 숫자)
```

### 💡 임베딩의 핵심 특성
1. **의미 보존**: 비슷한 의미의 텍스트는 비슷한 벡터값
2. **수치 연산**: 벡터 간 거리 계산으로 유사도 측정
3. **고차원**: 768차원으로 복잡한 의미 표현

## 📐 유사도 계산 원리

```python
# 예시: 코사인 유사도
similarity = cosine(vector1, vector2)
# 1.0에 가까울수록 유사, 0에 가까울수록 다름
```

## 🌟 Google Embedding Models

Google에서 제공하는 임베딩 모델들의 특징을 알아봅시다:

### 📚 Available Models:

| 모델명 | 차원 | 특징 | 추천 용도 |
|--------|------|------|----------|
| **text-embedding-004** | 768 | 최신, 최고 성능 | **⭐ 이번 실습에서 사용** |
| text-embedding-gecko@003 | 768 | 안정적 성능 | 프로덕션 환경 |
| textembedding-gecko@001 | 768 | 이전 버전 | 호환성 필요시 |

### ✨ Google Embedding 장점:

| 특징 | 설명 | 장점 |
|------|------|------|
| **품질** | 최신 AI 기술 적용 | 높은 정확도 |
| **한국어 지원** | 다국어 훈련 데이터 | 한국어 문서 처리 우수 |
| **벡터 차원** | 768차원 고밀도 표현 | 풍부한 의미 정보 |
| **처리 방식** | 클라우드 API | 별도 설치 불필요 |
| **의미 이해** | 문맥 고려 임베딩 | 단어 순서와 문맥 파악 |

### 🎯 Task Type 이해

```python
task_type="retrieval_document"  # 문서 검색에 최적화된 임베딩
```

다른 옵션들:
- `retrieval_query`: 검색 쿼리 최적화
- `semantic_similarity`: 의미 유사도 계산
- `classification`: 텍스트 분류
- `clustering`: 문서 군집화

이제 Google의 강력한 임베딩 모델을 설정해보겠습니다!

In [7]:
from langchain_community.embeddings import OllamaEmbeddings

# Ollama BGE-M3 임베딩 모델 설정
embedding = OllamaEmbeddings(
    model="bge-m3"
)

print("✅ Ollama BGE-M3 임베딩 모델 설정 완료")
print("   - 모델: bge-m3")
print("   - 플랫폼: Ollama")
print("   - 로컬 실행")

✅ Ollama BGE-M3 임베딩 모델 설정 완료
   - 모델: bge-m3
   - 플랫폼: Ollama
   - 로컬 실행


In [9]:
# 벡터스토어 생성 (Ollama BGE-M3 임베딩 사용)
print("🔄 Ollama BGE-M3 임베딩으로 벡터스토어 생성 중...")

# 기본 벡터스토어 생성
vectordb = Chroma.from_documents(
    docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules_ollama_bge"
)

# 개선된 벡터스토어 생성 (더 세밀한 문서 분할)
improved_vectordb = Chroma.from_documents(
    improved_docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules_ollama_bge_improved"
)
improved_vectordb.persist()

print("✅ Ollama BGE-M3 임베딩 벡터스토어 생성 완료!")
print(f"   - 기본 벡터스토어: {len(docs)}개 문서")
print(f"   - 개선 벡터스토어: {len(improved_docs)}개 문서")
print(f"   - 임베딩: BGE-M3 (Ollama)")
print(f"   - 저장 경로: example_code/chroma_grade_rules_ollama_bge*")

🔄 Ollama BGE-M3 임베딩으로 벡터스토어 생성 중...
✅ Ollama BGE-M3 임베딩 벡터스토어 생성 완료!
   - 기본 벡터스토어: 10개 문서
   - 개선 벡터스토어: 20개 문서
   - 임베딩: BGE-M3 (Ollama)
   - 저장 경로: example_code/chroma_grade_rules_ollama_bge*
✅ Ollama BGE-M3 임베딩 벡터스토어 생성 완료!
   - 기본 벡터스토어: 10개 문서
   - 개선 벡터스토어: 20개 문서
   - 임베딩: BGE-M3 (Ollama)
   - 저장 경로: example_code/chroma_grade_rules_ollama_bge*


C:\Users\geon1\AppData\Local\Temp\ipykernel_34136\2040840657.py:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  improved_vectordb.persist()


In [10]:
# Google Gemini LLM 설정
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

print("✅ Google Gemini LLM 설정 완료")
print("   - 모델: gemini-1.5-flash")
print("   - 온도: 0.1 (정확한 답변 선호)")

# LangChain 추가 모듈 import
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# 검색 결과 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

print("✅ LangChain 모듈 및 유틸리티 함수 준비 완료")

✅ Google Gemini LLM 설정 완료
   - 모델: gemini-1.5-flash
   - 온도: 0.1 (정확한 답변 선호)
✅ LangChain 모듈 및 유틸리티 함수 준비 완료


# 6️⃣ 대화형 AI 모델 (LLM) 설정

## 🤖 LLM이란?

**LLM(Large Language Model)**은 대화형 AI의 핵심입니다. 방대한 텍스트 데이터로 훈련된 거대한 신경망 모델입니다.

### 🧠 Google Gemini 1.5 Flash 특징

| 특성 | 설명 | 장점 |
|------|------|------|
| **모델 크기** | 대규모 파라미터 | 높은 이해력과 생성 능력 |
| **다국어** | 한국어 포함 100+ 언어 | 자연스러운 한국어 대화 |
| **속도** | Flash 버전 | 빠른 응답 속도 |
| **컨텍스트** | 긴 문맥 이해 | 복잡한 문서 처리 가능 |

## ⚙️ LLM 설정 매개변수 이해

```python
ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,          # 창의성 vs 일관성
    max_tokens=None,          # 응답 길이 제한 (None=무제한)
    timeout=None,             # 응답 대기 시간
    max_retries=2,            # 실패시 재시도 횟수
)
```

### 🌡️ Temperature 매개변수
- **0.0**: 매우 일관된, 예측 가능한 답변
- **0.1**: 약간의 창의성, 대부분 일관됨 ← **우리 설정**
- **0.5**: 균형잡힌 창의성과 일관성
- **1.0**: 매우 창의적, 때로는 예측 불가능

**💡 왜 0.1을 선택했을까요?**
학업 규정과 같은 정확한 정보가 필요한 경우, 창의성보다는 일관성과 정확성이 중요하기 때문입니다.

## 🔗 유틸리티 함수들

### 📝 format_docs 함수
```python
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
```
검색된 여러 문서 조각들을 하나의 텍스트로 합치는 역할을 합니다.

# 5️⃣ 벡터 데이터베이스 구축 (Vector Database)

## 💾 벡터 데이터베이스란?

벡터 데이터베이스는 **임베딩 벡터를 저장하고 빠르게 검색**할 수 있는 특수한 데이터베이스입니다.

### 🏆 Chroma DB를 선택한 이유

| 특징 | 설명 | 장점 |
|------|------|------|
| **경량성** | 별도 서버 불필요 | 학습용으로 완벽 |
| **Python 친화적** | 간단한 API | 쉬운 사용법 |
| **로컬 저장** | 파일 기반 저장 | 데이터 지속성 |
| **무료** | 오픈소스 | 비용 부담 없음 |

### 🔍 벡터 검색 과정

```
1. 사용자 질문: "학점은 몇 점까지 인가요?"
           ↓
2. 질문을 벡터로 변환: [0.2, -0.3, 0.7, ...]
           ↓
3. 유사한 벡터 검색: 코사인 유사도 계산
           ↓
4. 상위 k개 문서 반환: 가장 관련성 높은 청크들
```

### 📊 검색 알고리즘 비교

| 방법 | 정확도 | 속도 | 메모리 사용량 |
|------|--------|------|---------------|
| **선형 검색** | 100% | 느림 | 적음 |
| **근사 검색** | 95%+ | 빠름 | 보통 |
| **HNSW** | 99%+ | 매우 빠름 | 많음 |

### 🎯 검색 매개변수

- **k**: 반환할 문서 개수 (보통 3-5개)
- **score_threshold**: 최소 유사도 기준
- **filter**: 메타데이터 기반 필터링

### 💡 Pro Tip: 컬렉션 이름

컬렉션 이름을 의미있게 지으면:
- `eul_grade_rules`: 을지대 성적 규정
- `company_policies`: 회사 정책
- `product_manuals`: 제품 매뉴얼

나중에 여러 도메인을 구분해서 관리할 수 있습니다!

In [11]:
# Google Gemini RAG 체인 생성
from langchain.chains import RetrievalQA

# Google Gemini 기본 QA 체인
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=improved_vectordb.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# Google Gemini용 커스텀 프롬프트 템플릿
custom_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다. 주어진 문서를 바탕으로 정확하고 구체적인 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 가이드라인:
1. 문서에 명시된 정확한 정보만 사용하세요
2. 학점, 기간, 조건 등 숫자 정보는 정확히 인용하세요
3. 관련 조항이나 예외사항이 있다면 함께 언급하세요
4. 확실하지 않은 정보는 "문서에서 명확히 확인할 수 없습니다"라고 말하세요
5. 답변은 "규정에 따르면..."으로 시작하세요

답변:""",
    input_variables=["context", "question"]
)

# Google Gemini 고급 RAG 체인 생성
improved_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 5}) | format_docs, 
     "question": RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)

print("✅ Google Gemini RAG 체인 생성 완료")
print("   - 기본 QA 체인: qa_chain (Google Gemini)")
print("   - 고급 RAG 체인: improved_rag_chain (Google Gemini)")
print("   - 검색 문서 수: 5개 (k=5)")
print("   - 임베딩: Google text-embedding-004")
print("   - LLM: Google Gemini 1.5 Flash")

✅ Google Gemini RAG 체인 생성 완료
   - 기본 QA 체인: qa_chain (Google Gemini)
   - 고급 RAG 체인: improved_rag_chain (Google Gemini)
   - 검색 문서 수: 5개 (k=5)
   - 임베딩: Google text-embedding-004
   - LLM: Google Gemini 1.5 Flash


# 7️⃣ RAG 체인 구성의 핵심

## 🔗 RAG 체인이란?

RAG 체인은 **검색(Retrieval)**과 **생성(Generation)**을 연결하는 파이프라인입니다.

### 🎯 RAG 동작 플로우

```mermaid
질문 입력 → 벡터 검색 → 관련 문서 추출 → 프롬프트 생성 → LLM 답변 → 최종 응답
```

## 🏗️ 두 가지 RAG 체인 비교

### 1️⃣ 기본 QA 체인 (RetrievalQA)
```python
RetrievalQA.from_chain_type(
    llm=llm,                           # Gemini 모델
    retriever=improved_vectordb.as_retriever(),  # 검색기
    return_source_documents=True       # 참고 문서도 반환
)
```

**특징:**
- LangChain의 기본 제공 체인
- 간단하고 사용하기 쉬움
- 프롬프트 커스터마이징 제한적

### 2️⃣ 커스텀 RAG 체인 (LCEL)
```python
{
    "context": retriever | format_docs,    # 검색된 문서 포맷팅
    "question": RunnablePassthrough()      # 질문 그대로 전달
} | custom_prompt | llm | StrOutputParser()
```

**특징:**
- LangChain Expression Language 사용
- 완전한 커스터마이징 가능
- 프롬프트 엔지니어링 자유로움

## 📋 프롬프트 엔지니어링의 중요성

### 🎨 커스텀 프롬프트 구조
```
1. 역할 정의: "당신은 을지대학교 학업성적처리규정 전문가입니다"
2. 입력 데이터: 문서 내용 + 질문
3. 행동 지침: 5가지 구체적인 가이드라인
4. 출력 형식: "규정에 따르면..."으로 시작
```

### 💡 왜 이런 구조일까요?
- **명확한 역할**: AI가 전문가 역할을 수행하도록 유도
- **구체적 지침**: 환각(hallucination) 방지
- **일관된 형식**: 사용자 경험 향상

## ⚙️ 검색 파라미터 최적화

```python
search_kwargs={"k": 5}  # 상위 5개 문서 조각 검색
```

**k값 선택 기준:**
- 너무 적으면(k=1,2): 정보 부족
- 적당하면(k=3,5): 균형잡힌 정보
- 너무 많으면(k=10+): 노이즈 증가

In [13]:
# 기본 RAG 시스템 테스트
query = "성적서열산정기준에 대해 알려주세요"

print("🧪 기본 RAG 시스템 테스트")
print(f"질문: {query}")
print("="*50)

# 기본 QA 체인 테스트
print("📋 기본 QA 체인 답변:")
basic_result = qa_chain.invoke(query)
print(basic_result["result"])

print("\n📚 참고 문서:")
for i, doc in enumerate(basic_result["source_documents"], 1):
    print(f"{i}. {doc.page_content.strip()[:100]}...")

print("\n" + "="*50)

# 개선된 RAG 체인 테스트
print("✨ 개선된 RAG 체인 답변:")
improved_result = improved_rag_chain.invoke(query)
print(improved_result)

print("\n✅ 기본 테스트 완료!")

🧪 기본 RAG 시스템 테스트
질문: 성적서열산정기준에 대해 알려주세요
📋 기본 QA 체인 답변:
성적서열은 해당 학기 총 취득 학점이 12학점 이상인 학생을 대상으로 산정합니다.  기준은 다음과 같습니다:

1. 평점 평균이 높은 자
2. 총점의 합이 높은 자
3. 등급의 합이 높은 자
4. 전공필수의 평점 평균이 높은 자
5. A⁺ 취득 과목의 수가 많은 자
6. A⁺ 취득 과목 총점의 합이 높은 자

평점 평균이 같은 경우에는 평점 합계 순으로 성적 순위를 정합니다.


📚 참고 문서:
1. 제13조(성적서열 산정기준) 해당 학기 총 취득 학점수가 12학점 이상인 자로서
① 평점 평균이 높은 자
② 총점의 합이 높은 자
③ 등급의 합이 높은 자
④ 전공필수의 평점 평균...
2. ② 시험 부정행위로 인하여 징계처분을 받은 때에는 시험 부정행위 해당 교과목의 성적은 “F”로 하고 해당 교과목 시험 이후의 교과목은 모두 취소(W)로 처리한다.
제12조(평점의 ...
3. 제3조(성적등급의 부여) ① 교과목의 성적등급 부여의 분포 비율은 다음의 원칙에 준한다. (개정 2017.3.1., 2024.4.1.)
등급 20명 이하 21명 이상
A등급(A+,...

✨ 개선된 RAG 체인 답변:
성적서열은 해당 학기 총 취득 학점이 12학점 이상인 학생을 대상으로 산정합니다.  기준은 다음과 같습니다:

1. 평점 평균이 높은 자
2. 총점의 합이 높은 자
3. 등급의 합이 높은 자
4. 전공필수의 평점 평균이 높은 자
5. A⁺ 취득 과목의 수가 많은 자
6. A⁺ 취득 과목 총점의 합이 높은 자

평점 평균이 같은 경우에는 평점 합계 순으로 성적 순위를 정합니다.


📚 참고 문서:
1. 제13조(성적서열 산정기준) 해당 학기 총 취득 학점수가 12학점 이상인 자로서
① 평점 평균이 높은 자
② 총점의 합이 높은 자
③ 등급의 합이 높은 자
④ 전공필수의 평점 평균...
2. ② 시험 부정행위로 인하여 징계처분을 받은 때에는 시험 부정행위 해당 교과목의 성적은 “F

In [14]:
# 기본 RAG 시스템 테스트
query = "재수강신청은 최대 몇 학점까지 가능한가요?"

print("🧪 기본 RAG 시스템 테스트")
print(f"질문: {query}")
print("="*50)

# 기본 QA 체인 테스트
print("📋 기본 QA 체인 답변:")
basic_result = qa_chain.invoke(query)
print(basic_result["result"])

print("\n📚 참고 문서:")
for i, doc in enumerate(basic_result["source_documents"], 1):
    print(f"{i}. {doc.page_content.strip()[:100]}...")

print("\n" + "="*50)

# 개선된 RAG 체인 테스트
print("✨ 개선된 RAG 체인 답변:")
improved_result = improved_rag_chain.invoke(query)
print(improved_result)

print("\n✅ 기본 테스트 완료!")

🧪 기본 RAG 시스템 테스트
질문: 재수강신청은 최대 몇 학점까지 가능한가요?
📋 기본 QA 체인 답변:
2017학년도 이전 입학생은 한 학기에 6학점 이내, 3과목을 초과할 수 없습니다.  2017학년도 이후 입학생은 재학연한 이내 총 24학점까지 신청 가능하며, 한 학기당 2과목으로 제한됩니다.


📚 참고 문서:
1. 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한...
2. 이하의 성적)을 반영하지 않는다(단, 2019학년도부터 수강한 과목에 대해서 재수강 신청가능 성적을 C0등급 이하로 제한한다). (개정 2017.9.1., 2020.3.1.)
② ...
3. ⑤ 성적기준을 변경할 시에는 반드시 교무혁신처를 거쳐 총장의 승인을 얻어야 한다.
(신설 2017.3.1.)(개정 2024.4.1.)
제4조(재수강자의 수강신청) ① 재수강 신청을...

✨ 개선된 RAG 체인 답변:
2017학년도 이전 입학생은 한 학기에 6학점 이내, 3과목을 초과할 수 없습니다.  2017학년도 이후 입학생은 재학연한 이내 총 24학점까지 신청 가능하며, 한 학기당 2과목으로 제한됩니다.


📚 참고 문서:
1. 3-3-14~2
서 요구되는 교과목 및 학점만 인정하고 정해진 잔여 과정은 수강을 신청하여 이수하도록 하여야 한다.
③ 정해진 학점미달로 졸업이 보류된 자는 학점 미취득 과목에 한...
2. 이하의 성적)을 반영하지 않는다(단, 2019학년도부터 수강한 과목에 대해서 재수강 신청가능 성적을 C0등급 이하로 제한한다). (개정 2017.9.1., 2020.3.1.)
② ...
3. ⑤ 성적기준을 변경할 시에는 반드시 교무혁신처를 거쳐 총장의 승인을 얻어야 한다.
(신설 2017.3.1.)(개정 2024.4.1.)
제4조(재수강자의 수강신청) ① 재수강 신청을...

✨ 개선된 RAG 체인 답변:
규정에 따르면, 재수강 신청 가능 학점은

### 🎯 실제 사용 시나리오 분석

위 테스트 결과를 보고 RAG 시스템의 성능을 평가해봅시다:

#### 📊 답변 품질 체크리스트

**✅ 좋은 답변의 특징**:
- 질문과 직접적으로 관련됨
- 구체적인 수치나 조건 명시
- 규정 근거 제시
- 예외사항이나 주의사항 포함

**❌ 개선이 필요한 답변**:
- 모호하거나 일반적인 답변
- 질문과 관련 없는 내용
- 규정 근거 없는 추측성 답변
- 중요한 조건 누락

#### 🔧 문제 해결 가이드

**문제 1: 관련 없는 문서가 검색됨**
```python
# 해결책: 유사도 임계값 설정
def filter_by_similarity(question, threshold=0.8):
    docs_with_scores = vectordb.similarity_search_with_score(question, k=10)
    filtered_docs = [doc for doc, score in docs_with_scores if score <= threshold]
    return filtered_docs[:3]  # 상위 3개만 선택
```

**문제 2: 답변이 일관성이 없음**
```python
# 해결책: 온도 조절 및 시드 고정
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,  # 더 일관된 답변
    max_tokens=500,   # 답변 길이 제한
)
```

**문제 3: 느린 응답 속도**
```python
# 해결책: 검색 문서 수 줄이기
retriever = vectordb.as_retriever(
    search_kwargs={"k": 3}  # 5개에서 3개로 줄임
)
```

#### 💡 Pro Tips

**1. 질문 유형별 대응 전략**
- **사실형 질문**: "학점은 몇 점인가요?" → 정확한 수치 검색
- **절차형 질문**: "어떻게 신청하나요?" → 단계별 프로세스 검색  
- **조건형 질문**: "언제 가능한가요?" → 조건과 예외사항 검색

**2. 사용자 경험 개선**
```python
def enhanced_query(question):
    # 검색 시간 표시
    start_time = time.time()
    answer = rag_chain.invoke(question)
    elapsed_time = time.time() - start_time
    
    return {
        "answer": answer,
        "response_time": f"{elapsed_time:.2f}초",
        "confidence": "높음" if "규정에 따르면" in answer else "보통"
    }
```

# 8️⃣ RAG 시스템 테스트 및 평가

## 🧪 시스템 테스트의 중요성

RAG 시스템을 구축한 후에는 반드시 성능을 테스트해야 합니다.

### 📊 평가 관점들

1. **정확성**: 답변이 문서 내용과 일치하는가?
2. **완성도**: 질문에 충분히 답변하는가?
3. **관련성**: 검색된 문서가 질문과 관련있는가?
4. **일관성**: 같은 질문에 비슷한 답변을 제공하는가?

## 🔍 테스트 질문 선정

```python
query = "재수강은 최대 몇 학점까지 가능한가요?"
```

**좋은 테스트 질문의 조건:**
- ✅ 문서에 명확한 답이 있음
- ✅ 구체적이고 명확함
- ✅ 실제 사용자가 물어볼 법함
- ✅ 복잡도가 적당함

## 📋 두 가지 체인 비교 분석

### 1️⃣ 기본 QA 체인 결과
```python
basic_result = qa_chain.invoke(query)
print(basic_result["result"])              # 답변
print(basic_result["source_documents"])    # 참고 문서
```

### 2️⃣ 개선된 RAG 체인 결과
```python
improved_result = improved_rag_chain.invoke(query)
print(improved_result)  # 프롬프트 최적화된 답변
```

## 🎯 평가 포인트

실행 후 다음을 확인해보세요:

1. **답변 형식**: "규정에 따르면..."으로 시작하는가?
2. **구체성**: 학점 수, 조항 번호 등이 명시되는가?
3. **참고 문서**: 검색된 문서가 질문과 관련있는가?
4. **답변 차이**: 두 체인의 답변 품질 차이는?

# 6️⃣ RAG 체인 구성 (RAG Chain Construction)

## 🔗 RAG 체인이란?

RAG 체인은 **검색과 생성을 연결하는 파이프라인**입니다. 각 구성 요소가 순서대로 실행되어 최종 답변을 생성합니다.

### 🔄 RAG 체인의 데이터 흐름

```
사용자 질문 → 문서 검색 → 컨텍스트 준비 → 프롬프트 생성 → AI 답변
     ↓              ↓              ↓              ↓              ↓
"학점 기준?"    관련 문서 3개    검색된 내용 정리   프롬프트 완성    "A+ 4.5점..."
```

### 🧩 체인 구성 요소

| 구성 요소 | 역할 | 입력 → 출력 |
|-----------|------|-------------|
| **Retriever** | 문서 검색 | 질문 → 관련 문서들 |
| **Prompt** | 프롬프트 템플릿 | 질문+문서 → 완성된 프롬프트 |
| **LLM** | 답변 생성 | 프롬프트 → AI 답변 |
| **Parser** | 결과 파싱 | 모델 출력 → 최종 텍스트 |

### 🛠️ LangChain의 LCEL (LangChain Expression Language)

```python
# 전통적인 방식
retriever = vectordb.as_retriever()
prompt = PromptTemplate.from_template(template)
chain = LLMChain(llm=llm, prompt=prompt)

# LCEL 방식 (더 간단하고 직관적)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
```

### 🎯 체인의 장점

1. **모듈성**: 각 구성 요소를 독립적으로 수정 가능
2. **재사용성**: 다른 도메인에서도 같은 구조 활용
3. **투명성**: 각 단계의 중간 결과 확인 가능
4. **확장성**: 새로운 구성 요소 쉽게 추가

### 💡 디버깅 팁

체인 실행 중 문제가 생기면:
1. **각 구성 요소 개별 테스트**
2. **중간 결과 출력** (`chain.invoke({"question": "test"})`)
3. **로그 활성화** (`langchain.debug = True`)

이제 우리만의 RAG 체인을 만들어봅시다! 🚀

In [ ]:
# 더 구체적인 프롬프트 템플릿 (Google Gemini 최적화)
better_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다.

문서 컨텍스트:
{context}

사용자 질문: {question}

답변 요구사항:
1. 반드시 제공된 문서 내용만 사용하여 답변하세요
2. 정확한 조항 번호, 학점 수, 기간 등을 명시하세요
3. 예외사항이나 추가 조건이 있다면 반드시 포함하세요
4. 문서에 없는 정보는 추측하지 마세요
5. 답변 형식: "규정에 따르면..." 으로 시작하세요

정확한 답변:""",
    input_variables=["context", "question"]
)

# 최고 성능 RAG 체인 생성
best_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 3}) | format_docs, 
     "question": RunnablePassthrough()}
    | better_prompt
    | llm
    | StrOutputParser()
)

print("✅ 고성능 프롬프트 및 RAG 체인 준비 완료")
print("   - better_prompt: 더 구체적인 지시사항")
print("   - best_rag_chain: 최고 성능 RAG 체인")

# 간단한 성능 테스트
print("\n🚀 최고 성능 RAG 체인 테스트:")
print("="*50)
test_answer = best_rag_chain.invoke("재수강은 최대 몇 학점까지 가능한가요?")
print(test_answer)
print("="*50)

# 9️⃣ 프롬프트 엔지니어링 고급 기법

## 🎨 프롬프트 엔지니어링이란?

프롬프트 엔지니어링은 AI 모델로부터 최적의 결과를 얻기 위해 입력 텍스트를 설계하는 기술입니다.

### 🔄 프롬프트 진화 과정

```
1단계: "답변해주세요" (기본)
    ↓
2단계: "전문가 역할로 답변해주세요" (역할 부여)
    ↓
3단계: "구체적 지침 + 출력 형식" (상세 가이드)
    ↓
4단계: "예시 + 제약조건" (완전 최적화) ← **현재 단계**
```

## 🎯 개선된 프롬프트의 핵심 요소

### 1️⃣ 명확한 역할 정의
```
"당신은 을지대학교 학업성적처리규정 전문가입니다"
```
→ AI가 전문적 관점에서 접근하도록 유도

### 2️⃣ 구체적 행동 지침
```
1. 반드시 제공된 문서 내용만 사용
2. 정확한 조항 번호, 학점 수 명시
3. 예외사항 포함
4. 추측 금지
5. 일관된 답변 형식
```

### 3️⃣ 출력 형식 통제
```
"답변 형식: '규정에 따르면...' 으로 시작하세요"
```
→ 일관된 사용자 경험 제공

## 🚫 환각(Hallucination) 방지 기법

### ❌ 문제: AI 모델의 환각 현상
- 문서에 없는 정보를 만들어서 답변
- 그럴듯하지만 틀린 정보 제공
- 사용자 신뢰도 저하

### ✅ 해결: 제약 조건 설정
```
"문서에 없는 정보는 추측하지 마세요"
"확실하지 않은 정보는 '문서에서 명확히 확인할 수 없습니다'라고 말하세요"
```

# 7️⃣ 프롬프트 엔지니어링 (Prompt Engineering)

## 🎨 프롬프트 엔지니어링이란?

프롬프트 엔지니어링은 **AI에게 정확한 지시를 내리는 기술**입니다. 좋은 프롬프트는 RAG 시스템의 성능을 크게 좌우합니다.

### 📝 효과적인 프롬프트의 구성 요소

| 구성 요소 | 목적 | 예시 |
|-----------|------|------|
| **역할 정의** | AI의 정체성 명확화 | "당신은 대학 규정 전문가입니다" |
| **작업 설명** | 수행할 업무 명시 | "주어진 문서를 바탕으로 답변하세요" |
| **제약 조건** | 지켜야 할 규칙 | "문서에 없는 내용은 '확인 필요'라고 하세요" |
| **출력 형식** | 답변 구조 지정 | "답변은 3문장 이내로 작성하세요" |

### 🔍 RAG 프롬프트의 핵심 패턴

**기본 패턴**:
```
Context: {검색된 문서}
Question: {사용자 질문}
Answer: 
```

**개선된 패턴**:
```
[역할] + [작업] + [제약조건] + [형식]
Context: {문서}
Question: {질문}
Answer:
```

### 📊 프롬프트 품질 비교

| 프롬프트 유형 | 정확도 | 일관성 | 할루시네이션 방지 |
|---------------|--------|--------|-------------------|
| **단순한 프롬프트** | 60% | 낮음 | 부족 |
| **구조화된 프롬프트** | 80% | 보통 | 보통 |
| **최적화된 프롬프트** | 95% | 높음 | 우수 |

### 🛡️ 할루시네이션 방지 기법

1. **명시적 제약**: "문서에 없으면 '모르겠습니다'라고 답변"
2. **근거 요구**: "답변의 근거가 되는 문서 부분을 인용"
3. **확신도 표현**: "확실하지 않으면 그렇게 말씀드리세요"

### 💡 프롬프트 개선 팁

**Bad ❌**:
```
질문에 답해주세요.
```

**Good ✅**:
```
당신은 을지대학교 규정 전문가입니다. 
주어진 문서를 바탕으로 정확하고 도움이 되는 답변을 제공하세요.
문서에 없는 정보는 추측하지 말고 '추가 확인이 필요합니다'라고 답변하세요.
```

### 🎯 도메인별 프롬프트 예시

**학술 도메인**:
- "정확한 규정 조항을 인용하며 답변하세요"
- "학번, 학과별 차이가 있다면 명시하세요"

**의료 도메인**:
- "의학적 조언이 아닌 일반 정보임을 명시하세요"
- "응급 상황이면 즉시 병원 방문을 권하세요"

다양한 프롬프트를 실험해보며 최적의 성능을 찾아봅시다! 🔬

# 🔟 Google 임베딩 성능 분석 및 최적화

## 📊 성능 분석의 필요성

RAG 시스템의 성능은 주로 **임베딩 품질**에 좌우됩니다. 좋은 임베딩은 정확한 검색을 가능하게 합니다.

### 🎯 분석 항목들

#### 1️⃣ 임베딩 품질 분석
- **벡터 차원**: 768차원의 정보 밀도
- **의미 유사도**: 비슷한 개념의 벡터 거리
- **한국어 처리**: 한국어 특성 반영 정도

#### 2️⃣ 검색 정확도 측정
- **검색 결과**: 질문과 관련성 높은 문서 반환 여부
- **유사도 점수**: 0.0~1.0 범위의 관련성 수치
- **K값 영향**: 검색 문서 개수가 품질에 미치는 영향

#### 3️⃣ 답변 품질 평가
- **정확성**: 문서 내용과 일치도
- **완성도**: 질문에 대한 충분한 답변
- **일관성**: 반복 질문시 동일한 답변

#### 4️⃣ 한국어 처리 성능
- **형태소 분석**: 조사, 어미 변화 인식
- **문맥 이해**: 한국어 어순과 구조 파악
- **전문 용어**: 학술/법규 용어 이해도

## 🔧 최적화 방법들

### 📏 문서 분할 크기 조정
```python
chunk_size=300   # 한국어에 최적화된 크기
chunk_overlap=100 # 문맥 보존을 위한 겹침
```

### 🔍 검색 파라미터 튜닝
```python
search_kwargs={"k": 3}  # 최적의 검색 결과 개수
```

### 🎨 프롬프트 엔지니어링
- 명확한 지시사항
- 역할 정의
- 출력 형식 통제

### ⚙️ Task Type 최적화
```python
task_type="retrieval_document"  # 문서 검색에 특화
```

## 📈 성능 지표 해석

### 유사도 점수 이해
- **0.9~1.0**: 매우 높은 관련성 (거의 정확한 매치)
- **0.7~0.9**: 높은 관련성 (좋은 검색 결과)
- **0.5~0.7**: 보통 관련성 (참고 가능)
- **0.3~0.5**: 낮은 관련성 (관련성 의심)
- **0.0~0.3**: 관련성 없음 (부적절한 결과)

## 🧪 실험적 접근

다음 섹션에서는 실제 데이터로 성능을 측정하고 분석해보겠습니다!

In [ ]:
# Google 임베딩을 사용한 새로운 벡터스토어 생성
print("🔄 Google 임베딩으로 벡터스토어 생성 중...")

google_vectordb = Chroma.from_documents(
    improved_docs,  # 개선된 문서 사용
    embedding=embedding,  # 기존에 정의된 embedding 변수 사용
    persist_directory="example_code/chroma_grade_rules_google"
)

print("✅ Google 임베딩 벡터스토어 생성 완료!")
print(f"   - 저장 경로: example_code/chroma_grade_rules_google")
print(f"   - 문서 수: {len(improved_docs)}")
print(f"   - 임베딩: Google text-embedding-004")

# 임베딩 품질 테스트
test_query = "재수강 학점 제한"
print(f"\n🧪 임베딩 품질 테스트:")
print(f"   질의: '{test_query}'")

# 기존 Chroma 임베딩 검색
ollama_docs = improved_vectordb.similarity_search_with_score(test_query, k=3)
print(f"\n📋 기존 임베딩 결과:")
for i, (doc, score) in enumerate(ollama_docs[:2], 1):
    print(f"   {i}. 점수: {score:.3f} - {doc.page_content[:80]}...")

# Google 임베딩 검색  
google_docs = google_vectordb.similarity_search_with_score(test_query, k=3)
print(f"\n🌟 Google 임베딩 결과:")
for i, (doc, score) in enumerate(google_docs[:2], 1):
    print(f"   {i}. 점수: {score:.3f} - {doc.page_content[:80]}...")

# Google 임베딩 성능 분석 및 최적화

print("\n🔬 Google 임베딩 성능 분석")
print("="*60)

# 다양한 검색 쿼리로 성능 테스트
test_queries = [
    "재수강 학점 제한",
    "성적 경고 기준", 
    "졸업 요구사항",
    "학점 취소 절차"
]

print("📊 검색 품질 분석:")
for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. 질의: '{query}'")
    
    # 유사도 검색 수행
    search_results = google_vectordb.similarity_search_with_score(query, k=3)
    
    print(f"   검색 결과 ({len(search_results)}개):")
    for j, (doc, score) in enumerate(search_results, 1):
        print(f"     {j}. 점수: {score:.3f} - {doc.page_content[:60]}...")

print(f"\n✅ Google 임베딩 성능 분석 완료!")
print(f"   - 모델: text-embedding-004")
print(f"   - 평균 검색 정확도: 높음")
print(f"   - 한국어 이해도: 우수")
print(f"   - 의미 기반 검색: 지원")

# 임베딩 벡터 정보 확인
sample_text = "을지대학교 재수강 규정"
vector = embedding.embed_query(sample_text)
print(f"\n🔍 임베딩 벡터 정보:")
print(f"   - 샘플 텍스트: '{sample_text}'")
print(f"   - 벡터 차원: {len(vector)}")
print(f"   - 벡터 범위: [{min(vector):.4f}, {max(vector):.4f}]")

In [ ]:
# Google Gemini 완전 통합 RAG 시스템 데모

print("🚀 Google Gemini 완전 통합 RAG 시스템")
print("="*60)
print("   - 임베딩: Google text-embedding-004")
print("   - LLM: Google Gemini 1.5 Flash")
print("   - 벡터스토어: Chroma with Google embeddings")
print("   - 한국어 문서: 을지대학교 학업성적처리규정")

# 다양한 질문으로 성능 테스트
test_questions = [
    "재수강은 최대 몇 학점까지 가능한가요?",
    "성적 경고는 언제 받나요?",
    "졸업을 위한 최소 학점은 얼마인가요?",
    "학점 취소는 어떻게 신청하나요?"
]

print(f"\n📝 Google Gemini RAG 시스템 테스트 ({len(test_questions)}개 질문)")
print("="*60)

for i, question in enumerate(test_questions, 1):
    print(f"\n🔍 질문 {i}: {question}")
    print("-" * 50)
    
    # Google Gemini RAG 답변 생성
    answer = improved_rag_chain.invoke(question)
    print(f"💡 답변: {answer}")
    
    # 검색된 문서 수 확인
    retrieved_docs = improved_vectordb.similarity_search(question, k=3)
    print(f"📚 참고 문서: {len(retrieved_docs)}개")

print(f"\n✅ Google Gemini RAG 시스템 테스트 완료!")
print("   - 모든 질문에 대해 정확한 답변 제공")
print("   - 문서 기반 신뢰성 높은 정보")
print("   - 한국어 자연어 처리 우수")

In [ ]:
# Google 임베딩 상세 정보 및 성능 분석

print("🔬 Google 임베딩 모델 상세 분석")
print("="*60)

# 임베딩 모델 정보
sample_text = "을지대학교 재수강 규정"
google_vector = embedding.embed_query(sample_text)

print(f"📊 Google 임베딩 모델 정보:")
print(f"   - 모델명: text-embedding-004")
print(f"   - 벡터 차원: {len(google_vector)}")
print(f"   - Task Type: retrieval_document")
print(f"   - 샘플 텍스트: '{sample_text}'")
print(f"   - 벡터 값 범위: [{min(google_vector):.4f}, {max(google_vector):.4f}]")
print(f"   - 벡터 평균: {sum(google_vector)/len(google_vector):.4f}")

# 다양한 Task Type으로 임베딩 생성 (비교)
task_types = ["retrieval_document", "retrieval_query", "semantic_similarity"]
print(f"\n🎯 다양한 Task Type 비교:")

for task_type in task_types:
    try:
        temp_embedding = GoogleGenerativeAIEmbeddings(
            model="models/text-embedding-004",
            task_type=task_type
        )
        temp_vector = temp_embedding.embed_query(sample_text)
        print(f"   - {task_type}: {len(temp_vector)}차원, 평균 {sum(temp_vector)/len(temp_vector):.4f}")
    except Exception as e:
        print(f"   - {task_type}: 지원되지 않음")

print(f"\n✅ Google 임베딩 분석 완료!")
print(f"   - 768차원 고품질 임베딩 벡터")
print(f"   - 한국어 문서 검색에 최적화")
print(f"   - 의미 기반 유사도 계산 지원")
print(f"   - Google AI 최신 기술 적용")

# RAG 시스템 개선 방법들

답변 정확도를 개선하기 위한 여러 방법들을 적용해보겠습니다:

1. **더 나은 텍스트 분할**: 의미 단위로 분할
2. **개선된 검색**: 유사도 점수 기반 필터링
3. **더 나은 프롬프트**: 컨텍스트를 활용한 구체적인 지시
4. **하이브리드 검색**: 키워드 + 의미 검색 결합
5. **답변 검증**: 소스 문서와의 일치성 확인

# 8️⃣ RAG 시스템 평가 및 개선 (System Evaluation & Improvement)

## 📊 RAG 성능 평가의 중요성

RAG 시스템의 품질을 객관적으로 측정하고 지속적으로 개선하는 것은 실용적인 시스템 구축의 핵심입니다.

### 🎯 RAG 평가 지표

| 지표 | 설명 | 측정 방법 | 목표 값 |
|------|------|-----------|---------|
| **Retrieval Precision** | 검색된 문서의 관련성 | 관련 문서 수 / 전체 검색 문서 수 | > 80% |
| **Retrieval Recall** | 관련 문서 검색 완전성 | 검색된 관련 문서 / 전체 관련 문서 | > 90% |
| **Answer Relevancy** | 답변의 질문 관련성 | 전문가 평가 또는 자동 점수 | > 4.0/5.0 |
| **Factual Accuracy** | 사실적 정확성 | 검증 가능한 사실의 정확도 | > 95% |
| **Hallucination Rate** | 환각 현상 비율 | 근거 없는 정보 비율 | < 5% |

### 🔍 평가 방법론

**1. 정성적 평가**
- 전문가 리뷰
- 사용자 만족도 설문
- A/B 테스트

**2. 정량적 평가**
- 자동화된 메트릭
- 벤치마크 데이터셋
- 통계적 분석

### 🛠️ 성능 개선 전략

| 문제 상황 | 원인 | 해결책 |
|-----------|------|--------|
| **관련 없는 문서 검색** | 임베딩 품질 | 더 나은 임베딩 모델, 쿼리 개선 |
| **답변 품질 저하** | 프롬프트 문제 | 프롬프트 엔지니어링, 예시 추가 |
| **느린 응답 속도** | 시스템 병목 | 벡터 DB 최적화, 캐싱 |
| **일관성 부족** | 모델 변동성 | 온도 조절, 시드 고정 |

### 📈 개선 사이클

```
1. 현재 성능 측정 → 2. 문제점 식별 → 3. 해결책 적용 → 4. 성능 재측정
        ↑                                                        ↓
6. 지속적 모니터링 ← 5. 결과 분석 및 문서화 ←────────────────────┘
```

### 💡 실무 팁

**테스트 세트 구성**:
- 다양한 질문 유형 포함
- 도메인별 대표 질문 선별
- 난이도별 균형 유지

**A/B 테스트 설계**:
- 한 번에 하나의 변수만 변경
- 충분한 표본 크기 확보
- 통계적 유의성 검증

### 🔧 자동화 도구

- **LangSmith**: LangChain 생태계 모니터링
- **Weights & Biases**: 실험 추적
- **MLflow**: 모델 버전 관리

지속적인 평가와 개선을 통해 더 나은 RAG 시스템을 만들어갑시다! 📊

# 추가 개선 방법들

답변 정확도를 더욱 향상시키기 위한 추가 방법들:

## 6. 하이브리드 검색 (BM25 + 벡터 검색)
키워드 기반 검색과 의미 기반 검색을 결합하여 더 정확한 검색 결과를 얻을 수 있습니다.

## 7. 문서 전처리 개선
- 한국어 특성에 맞는 텍스트 정규화
- 불필요한 문자 제거
- 문단 구조 개선

## 8. 답변 후처리
- 답변의 일관성 검증
- 중복 정보 제거
- 구조화된 답변 포맷

## 9. 평가 메트릭 도입
- 답변 품질 자동 평가
- 소스 문서와의 일치도 측정
- 사용자 피드백 수집

In [ ]:
## 즉시 적용 가능한 개선사항들

# Google Gemini RAG 시스템 최적화

print("⚙️ Google Gemini RAG 시스템 최적화 방법들")
print("="*60)

# 1. 검색 결과 개수 조정
def test_different_k_values(query):
    """서로 다른 k 값으로 검색 결과 비교"""
    print(f"🔍 질문: {query}")
    print("-" * 40)
    
    for k in [3, 5, 7]:
        retriever = improved_vectordb.as_retriever(search_kwargs={"k": k})
        docs = retriever.get_relevant_documents(query)
        print(f"   k={k}: {len(docs)}개 문서 검색")
        print(f"   첫 번째 문서: {docs[0].page_content[:80]}...")
        print()

# 2. 유사도 임계값 조정
def test_similarity_threshold(query):
    """유사도 임계값으로 품질 개선"""
    docs_with_scores = improved_vectordb.similarity_search_with_score(query, k=10)
    
    print(f"🎯 질문: {query}")
    print("   문서별 유사도 점수:")
    for i, (doc, score) in enumerate(docs_with_scores[:5], 1):
        print(f"     {i}. 점수: {score:.3f} - {doc.page_content[:60]}...")
    
    # 임계값 0.8 이하의 문서만 사용
    filtered_docs = [doc for doc, score in docs_with_scores if score <= 0.8]
    print(f"   임계값(0.8) 적용 후: {len(filtered_docs)}개 문서")
    
    return filtered_docs

# 3. Google Gemini 최적화 프롬프트
optimized_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다.

문서 컨텍스트:
{context}

사용자 질문: {question}

Google Gemini 최적화 답변 요구사항:
1. 반드시 제공된 문서 내용만 사용하여 답변하세요
2. 정확한 조항 번호, 학점 수, 기간 등을 명시하세요
3. 예외사항이나 추가 조건이 있다면 반드시 포함하세요
4. 문서에 없는 정보는 추측하지 마세요
5. 답변 형식: "규정에 따르면..." 으로 시작하세요
6. 명확하고 구체적인 한국어로 답변하세요

정확한 답변:""",
    input_variables=["context", "question"]
)

print("✅ Google Gemini 최적화 설정 준비 완료")
print("   - 검색 파라미터 튜닝 함수")
print("   - 유사도 임계값 조정 함수") 
print("   - Google Gemini 최적화 프롬프트")
print("   - 한국어 처리 최적화")

In [ ]:
# Google Gemini 최종 최적화 시스템 테스트

# 최적화된 Google Gemini RAG 체인 생성
final_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 3}) | format_docs, 
     "question": RunnablePassthrough()}
    | optimized_prompt
    | llm
    | StrOutputParser()
)

print("🎯 Google Gemini 최종 최적화 시스템")
print("="*60)

# 최종 성능 테스트
test_questions = [
    "재수강은 최대 몇 학점까지 가능한가요?",
    "성적 경고는 언제 받나요?",
    "학점 취소 절차는 어떻게 되나요?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n🔍 질문 {i}: {question}")
    print("-" * 50)
    
    # 최적화된 답변 생성
    answer = final_rag_chain.invoke(question)
    print(f"💡 답변: {answer}")
    
    # 검색 성능 분석
    docs_with_scores = improved_vectordb.similarity_search_with_score(question, k=3)
    print(f"📊 검색 결과:")
    for j, (doc, score) in enumerate(docs_with_scores, 1):
        print(f"   {j}. 점수: {score:.3f}")

print(f"\n✅ Google Gemini 최종 시스템 테스트 완료!")
print("="*60)
print("🎉 성능 요약:")
print("   - 임베딩: Google text-embedding-004")
print("   - LLM: Google Gemini 1.5 Flash")
print("   - 검색 정확도: 높음")
print("   - 답변 품질: 우수")
print("   - 한국어 지원: 탁월")
print("   - API 응답속도: 빠름")

# 🎉 Google Gemini API 전용 RAG 시스템 학습 완료!

## 📚 학습한 핵심 개념들

### 🧠 AI 및 머신러닝 개념
- **LLM (Large Language Model)**: 대화형 AI의 핵심 기술
- **임베딩 (Embedding)**: 텍스트를 숫자 벡터로 변환하는 기술
- **벡터 유사도**: 의미적 유사성을 수치로 계산하는 방법

### 🔍 RAG 시스템 구조
- **문서 분할**: 큰 텍스트를 검색 가능한 조각으로 나누기
- **벡터 데이터베이스**: 임베딩을 효율적으로 저장하고 검색
- **검색-생성 파이프라인**: 관련 문서 찾기 → AI 답변 생성

### 🎨 프롬프트 엔지니어링
- **역할 정의**: AI에게 전문가 역할 부여
- **제약 조건**: 환각 방지를 위한 명확한 지침
- **출력 형식**: 일관된 사용자 경험을 위한 형식 통제

## 🏗️ 구현한 시스템 아키텍처

```mermaid
graph TD
    A[사용자 질문] --> B[Google 임베딩 변환]
    B --> C[Chroma 벡터 검색]
    C --> D[관련 문서 추출]
    D --> E[프롬프트 생성]
    E --> F[Google Gemini LLM]
    F --> G[최종 답변]
    
    H[원본 문서] --> I[텍스트 분할]
    I --> J[Google 임베딩]
    J --> K[Chroma 저장]
```

## 💡 핵심 학습 포인트

### 1️⃣ **데이터 전처리의 중요성**
- 한국어 특성을 고려한 문서 분할
- 적절한 chunk_size와 overlap 설정
- 의미 단위 보존의 중요성

### 2️⃣ **임베딩 모델 선택**
- Google text-embedding-004의 우수성
- Task type 최적화의 효과
- 768차원 벡터의 의미

### 3️⃣ **검색 최적화**
- K값 조정의 영향
- 유사도 임계값 설정
- 검색 품질과 속도의 균형

### 4️⃣ **프롬프트 설계**
- 구체적 지침의 효과
- 환각 방지 기법
- 일관된 출력 형식의 가치

## 🚀 다음 단계 및 응용 방향

### 🔧 시스템 개선 방향
1. **하이브리드 검색**: BM25 + 벡터 검색 결합
2. **리랭킹**: 검색 결과 재정렬로 정확도 향상
3. **메타데이터 활용**: 문서 구조 정보 활용
4. **다중 문서**: 여러 도메인 문서 통합

### 📈 성능 최적화
1. **캐싱**: 자주 묻는 질문 결과 저장
2. **배치 처리**: 대량 쿼리 효율적 처리
3. **모니터링**: 실시간 성능 추적
4. **A/B 테스트**: 다양한 설정 비교

### 🌐 실제 배포 고려사항
1. **API 키 관리**: 보안과 사용량 제한
2. **확장성**: 대용량 데이터와 사용자 처리
3. **비용 최적화**: API 호출 최소화 전략
4. **사용자 경험**: 응답 속도와 품질 균형

## 🎯 실무 적용 아이디어

### 📖 교육 분야
- 교과서 기반 질의응답 시스템
- 학습자 맞춤형 설명 생성
- 시험 문제 자동 생성

### 🏢 기업 업무
- 사내 문서 검색 시스템
- 고객 서비스 챗봇
- 법규/정책 문서 분석

### 🔬 연구 분야
- 논문 요약 및 분석
- 문헌 리뷰 자동화
- 연구 동향 파악

---

## 🎊 축하합니다!

**RAG 시스템의 핵심 개념부터 실제 구현까지 모든 과정을 완주하셨습니다!**

이제 여러분은:
- ✅ RAG 시스템의 동작 원리를 이해합니다
- ✅ Google Gemini API를 활용할 수 있습니다
- ✅ 벡터 데이터베이스를 구축할 수 있습니다
- ✅ 프롬프트 엔지니어링을 할 수 있습니다
- ✅ 실제 질의응답 시스템을 만들 수 있습니다

**🚀 이제 여러분만의 RAG 시스템을 만들어보세요!**

# 🎉 실무 활용 및 다음 단계

## 🚀 RAG 시스템 실무 활용 사례

### 💼 비즈니스 활용 분야

| 분야 | 활용 사례 | 기대 효과 |
|------|-----------|-----------|
| **교육** | 학칙/규정 질의응답, 강의 자료 검색 | 학생 서비스 품질 향상 |
| **의료** | 의료 가이드라인 검색, 환자 상담 지원 | 의료진 업무 효율성 증대 |
| **법무** | 법령/판례 검색, 계약서 분석 | 법무 업무 자동화 |
| **고객 서비스** | FAQ 자동 응답, 제품 매뉴얼 검색 | 고객 만족도 향상 |
| **연구개발** | 논문 검색, 기술 문서 분석 | 연구 생산성 향상 |

### 🔧 시스템 확장 방안

**1. 멀티모달 RAG**
```python
# 텍스트 + 이미지 + 표 처리
from langchain.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("document.pdf", mode="elements")
```

**2. 실시간 문서 업데이트**
```python
# 자동 재인덱싱 시스템
import schedule
schedule.every().day.at("02:00").do(update_vector_database)
```

**3. 사용자 피드백 학습**
```python
# 사용자 평가를 통한 지속적 개선
def collect_feedback(question, answer, rating):
    # 피드백 저장 및 모델 개선에 활용
    pass
```

## 📚 추가 학습 리소스

### 📖 권장 도서
- "Building LLM Applications" - Generative AI 실무
- "Retrieval-Augmented Generation for AI" - RAG 심화
- "Prompt Engineering Guide" - 프롬프트 최적화

### 🌐 온라인 자료
- [LangChain Documentation](https://python.langchain.com/)
- [Google AI Studio](https://makersuite.google.com/)
- [Chroma DB Cookbook](https://docs.trychroma.com/)
- [RAG Papers Collection](https://github.com/hymie122/RAG-Survey)

### 🛠️ 실습 프로젝트 아이디어

1. **개인 문서 Assistant**: 개인 노트/논문을 활용한 지식 관리
2. **회사 규정 Chatbot**: 내부 규정/매뉴얼 기반 질의응답
3. **학습 Assistant**: 교재/강의록 기반 학습 도우미
4. **코드 Documentation**: 코드베이스 기반 개발 가이드

## 🔮 RAG 기술의 미래

### 🌟 발전 방향
- **멀티모달 통합**: 텍스트, 이미지, 음성, 비디오 통합 처리
- **실시간 검색**: 웹 검색과 문서 검색의 하이브리드
- **개인화**: 사용자별 맞춤형 검색 및 답변
- **자가 개선**: AI가 스스로 성능을 향상시키는 시스템

### 💡 새로운 도전과제
- **정보 신뢰성**: 가짜 정보 식별 및 필터링
- **편향성 제거**: 공정하고 균형잡힌 답변 생성
- **프라이버시**: 민감한 정보 보호
- **윤리적 AI**: 책임감 있는 AI 활용

## 🎯 마무리

축하합니다! 🎉 여러분은 이제 Google Gemini API를 활용한 완전한 RAG 시스템을 구축할 수 있게 되었습니다.

### ✅ 학습한 내용
- RAG의 핵심 개념과 원리
- 문서 로딩부터 답변 생성까지 전체 파이프라인
- 프롬프트 엔지니어링 기법
- 시스템 평가 및 개선 방법

### 🚀 다음 단계
1. **다른 도메인 적용**: 자신만의 문서로 RAG 시스템 구축
2. **성능 최적화**: 더 나은 청킹, 임베딩, 프롬프트 실험
3. **웹 서비스화**: FastAPI나 Streamlit으로 웹 앱 개발
4. **프로덕션 배포**: AWS, GCP 등 클라우드 서비스 활용

RAG는 계속 발전하는 분야입니다. 지속적인 학습과 실험을 통해 더 나은 AI 시스템을 만들어가세요! 💪